In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Ruta absoluta al directorio principal del proyecto
proyecto_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Agregar la ruta al sistema
if proyecto_path not in sys.path:
    sys.path.append(proyecto_path)

In [2]:
from src.database import QdrantSingleton

client = QdrantSingleton(local=True).get_client()
client

In [ ]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

In [6]:
from qdrant_client.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ],
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [7]:
client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=False,
    limit=3
).points

[ScoredPoint(id=4, version=0, score=1.362, payload=None, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1, version=0, score=1.273, payload=None, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=3, version=0, score=1.208, payload=None, vector=None, shard_key=None, order_value=None)]

In [14]:
type(point.vector)

NoneType

In [17]:
point

Record(id='00b8b1a7-56ab-58e3-86d3-f36bca64ec3b', payload={'WEB_PROPERTY_CODE': 192861874, 'PRICE': 1372000, 'PRICE_ADMIN_INCLUDED': 1650000, 'AREA': 80.0, 'LATITUDE': 4.714549004433, 'LONGITUDE': -74.115074879543, 'ANTIQUITY': 'Más de 30 años', 'CONSTRUCTION_YEAR': None, 'BUILT_AREA': 80.0, 'PRIVATE_AREA': 0.0, 'GARAGE': 0, 'BATHROOMS': 1, 'ROOMS': 3, 'FLOOR': 1, 'PROPERTY_TYPE': 'Apartamento', 'OPERATION_TYPE': 'Arriendo', 'STRATUM': 3, 'BEDROOMS': 3, 'LINK': 'www.fincaraiz.com.co/apartamento-en-arriendo-en-bolivia-bogota/192861874'}, vector=None, shard_key=None, order_value=None)

In [19]:
point.id

'00b8b1a7-56ab-58e3-86d3-f36bca64ec3b'

In [20]:
from qdrant_client.models import UpdateVectors, PointStruct

# Get all points with scroll
offset = None
while True:
    result = client.scroll(
        collection_name="Arriendo",
        limit=100,
        offset=offset
    )
    
    points, next_offset = result
    
    if not points:
        break
        
    # Update points with corrected LINK
    updates = []
    for point in points:
        if 'LINK' in point.payload and 'ficnaraiz' in point.payload['LINK']:
            corrected_link = point.payload['LINK'].replace('ficnaraiz', 'fincaraiz')
            point.payload['LINK'] = corrected_link
            
            try:
                updates.append(PointStruct(
                    id=point.id,
                    vector=point.vector,
                    payload=point.payload
                ))
            except:
                pass
    
    # Upsert corrected points
    if len(updates) > 0:
        client.upsert(
            collection_name="Arriendo",
            points=updates
        )
    
    offset = next_offset
    if offset is None:
        break

In [3]:
from qdrant_client.models import PayloadSchemaType

# Create indexes for filtering fields
client = QdrantSingleton(local=False).get_client()

indexes_to_create = [
    ("BUILT_AREA", PayloadSchemaType.FLOAT),
    ("GARAGE", PayloadSchemaType.INTEGER), 
    ("BATHROOMS", PayloadSchemaType.INTEGER),
    ("ROOMS", PayloadSchemaType.INTEGER)
]

for field_name, schema_type in indexes_to_create:
    try:
        client.create_payload_index(
            collection_name="Arriendo",
            field_name=field_name,
            field_schema=schema_type
        )
        print(f"✅ Index created for {field_name}")
    except Exception as e:
        print(f"❌ Error creating index for {field_name}: {e}")

✅ Index created for BUILT_AREA
✅ Index created for GARAGE
✅ Index created for BATHROOMS
✅ Index created for ROOMS


In [ ]:
from qdrant_client.models import Filter, FieldCondition, Range
xd = {"must":["FieldCondition(key='BUILT_AREA', match=None, range=Range(lt=None, gt=None, gte=30.0, lte=60.0), geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None, is_empty=None, is_null=None)","FieldCondition(key='GARAGE', match=None, range=Range(lt=None, gt=None, gte=2.0, lte=3.0), geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None, is_empty=None, is_null=None)","FieldCondition(key='BATHROOMS', match=None, range=Range(lt=None, gt=None, gte=2.0, lte=3.0), geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None, is_empty=None, is_null=None)","FieldCondition(key='ROOMS', match=None, range=Range(lt=None, gt=None, gte=2.0, lte=3.0), geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None, is_empty=None, is_null=None)"]}
# Test filtering - will fail if no index exists
try:
    result = client.scroll(
        collection_name="Arriendo",
        scroll_filter=Filter(**xd),
        limit=1
    )
    print("✅ BUILT_AREA index works - filtering successful")
except Exception as e:
    print(f"❌ Index issue: {e}")

In [38]:
# Filtros generales
keys = ["BUILT_AREA", "GARAGE", "BATHROOMS", "ROOMS", "FLOOR"]
ranges = []
ranges.append([44,45])
ranges.append([0,1])
ranges.append([0,1])
ranges.append([0,2])
ranges.append([4,67])
must_condition = [FieldCondition(key=key, range=Range(gte=range_tuple[0], lte=range_tuple[1]))
    for key, range_tuple in zip(keys, ranges)]
query_payload = {
        'must':must_condition
    }

client = QdrantSingleton(local=False).get_client()
import ollama
vector = ollama.embed(model='nomic-embed-text', input=['momarods'])['embeddings'][0]


search_result = client.query_points(
collection_name="Arriendo",
query=vector,
query_filter = Filter(**query_payload),
with_payload=True,
limit=12
).points

In [40]:
client = QdrantSingleton(local=False).get_client()
collection_info = client.get_collection("Arriendo")
print("Existing indexes:", list(collection_info.payload_schema.keys()))

Existing indexes: ['GARAGE', 'BUILT_AREA', 'BATHROOMS', 'ROOMS']


In [32]:
# Run this in your notebook connected to cloud
from src.database import QdrantSingleton
from qdrant_client.models import PayloadSchemaType

client = QdrantSingleton(local=False).get_client()  # Cloud connection

for field_name, schema_type in [
    ("BUILT_AREA", PayloadSchemaType.FLOAT),
    ("GARAGE", PayloadSchemaType.INTEGER), 
    ("BATHROOMS", PayloadSchemaType.INTEGER),
    ("ROOMS", PayloadSchemaType.INTEGER)
]:
    client.create_payload_index(
        collection_name="Arriendo",
        field_name=field_name,
        field_schema=schema_type
    )

In [22]:
# Debug your vector format
print(f"Vector type: {type(vector)}")
print(f"Vector shape: {len(vector)} x {len(vector[0]) if vector else 'None'}")

# Try flattening if it's nested
if isinstance(vector, list) and isinstance(vector[0], list):
    vector = vector[0]  # Take first embedding

Vector type: <class 'list'>
Vector shape: 1 x 768


In [24]:
# Ensure single vector format
vector = ollama.embed(model='nomic-embed-text', input=['momarods'])['embeddings'][0]

# Verify it's a flat list of floats
print(f"Vector length: {len(vector)}")
print(f"First few values: {vector[:5]}")

Vector length: 768
First few values: [0.05826387, 0.007860063, -0.14971018, 0.011191034, -0.014298193]
